In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [ ]:
# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.rozee.pk/")
time.sleep(3)

women_link = driver.find_element(By.XPATH, '//a[@href="/collections/women-apparel"]')
women_link.click()
time.sleep(5)

product_names = []
product_prices = []
product_links = []

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Wait for new products to load

    # Check for new products
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # If no new content loaded, break the loop
        break
    last_height = new_height

# Step 3: Find all product containers using div.card
products = driver.find_elements(By.CLASS_NAME, "card")
for product in products:
    if not product.is_displayed():  # Skip products that are not visible
        continue
    # Locate the product name and link from the anchor tag inside div.card-product
    link_element = product.find_element(By.CSS_SELECTOR, "a.card-link")

    product_name = link_element.get_attribute("title")
    product_link = "https://www.rozee.pk" + link_element.get_attribute("href")
    
    
    try:
        # Check for sale price first
        sale_price = product.find_element(By.CLASS_NAME, "price-item--sale").text.strip()
    except Exception:
        sale_price = None

    try:
        # Check for regular price if no sale price is available
        if sale_price:
            price = sale_price
        else:
            price = product.find_element(By.CLASS_NAME, "price-item--regular").text.strip()
    except Exception:
            # If no regular price is available, check for another price structure
            try:
                price = product.find_element(By.CLASS_NAME, "price__regular").text.strip()
            except Exception:
                price = "Price not available"

        # Store collected data
    product_names.append(product_name)
    product_prices.append(price)
    product_links.append(product_link)
    
Final=pd.DataFrame({"Product Title": product_names,
                    "Price": product_prices,
                    "Links":product_links})